In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# Formato do caminho_arquivo: 'C:/exemplo/exemplo/exemplo/exemplo/'
caminho_arquivo = 'C:/exemplo/exemplo/exemplo/exemplo/'
caminho_base_cadastral = f'{caminho_arquivo}base_cadastral.csv'
caminho_base_info = f'{caminho_arquivo}base_info.csv'
caminho_base_pagamentos_desenvolvimento = f'{caminho_arquivo}base_pagamentos_desenvolvimento.csv'
caminho_base_pagamentos_teste = f'{caminho_arquivo}base_pagamentos_teste.csv'

df_cadastral = pd.read_csv(caminho_base_cadastral, sep=';', encoding='utf-8')
df_info = pd.read_csv(caminho_base_info, sep=';', encoding='utf-8')
df_pagamentos_desenvolvimento = pd.read_csv(caminho_base_pagamentos_desenvolvimento, sep=';', encoding='utf-8')
df_pagamentos_teste = pd.read_csv(caminho_base_pagamentos_teste, sep=';', encoding='utf-8')

In [3]:
# ANÁLISE EXPLORATÓRIA E PRÉ-PROCESSAMENTO (df_cadastral, df_info, df_pagamentos_desenvolvimento)
df_cadastral.head()

,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG
0,1661240395903230676,2013-08-22,99,NaN,Serviços,YAHOO,PEQUENO,65
1,8274986328479596038,2017-01-25,31,NaN,Comércio,YAHOO,MEDIO,77
2,345447888460137901,2000-08-15,75,NaN,Serviços,HOTMAIL,PEQUENO,48
3,1003144834589372198,2017-08-06,49,NaN,Serviços,OUTLOOK,PEQUENO,89
4,324916756972236008,2011-02-14,88,NaN,Serviços,GMAIL,GRANDE,62


In [4]:
df_cadastral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315 entries, 0 to 1314
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_CLIENTE           1315 non-null   int64 
 1   DATA_CADASTRO        1315 non-null   object
 2   DDD                  1078 non-null   object
 3   FLAG_PF              66 non-null     object
 4   SEGMENTO_INDUSTRIAL  1232 non-null   object
 5   DOMINIO_EMAIL        1285 non-null   object
 6   PORTE                1274 non-null   object
 7   CEP_2_DIG            1315 non-null   object
dtypes: int64(1), object(7)
memory usage: 82.3+ KB


In [5]:
# não vou usar DDD por não ser específica geograficamente, FLAG_PF ser desbalanceada
df_cadastral.drop(['DDD','FLAG_PF'], axis=1, inplace=True)

df_cadastral['CEP_2_DIG'] = pd.to_numeric(df_cadastral['CEP_2_DIG'], errors='coerce').astype('Int64')
df_cadastral['DATA_CADASTRO'] = pd.to_datetime(df_cadastral['DATA_CADASTRO'])

In [6]:
df_cadastral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315 entries, 0 to 1314
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_CLIENTE           1315 non-null   int64         
 1   DATA_CADASTRO        1315 non-null   datetime64[ns]
 2   SEGMENTO_INDUSTRIAL  1232 non-null   object        
 3   DOMINIO_EMAIL        1285 non-null   object        
 4   PORTE                1274 non-null   object        
 5   CEP_2_DIG            1314 non-null   Int64         
dtypes: Int64(1), datetime64[ns](1), int64(1), object(3)
memory usage: 63.1+ KB


In [7]:
df_info.head()

,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS
0,1661240395903230676,2018-09,16913.0,NaN
1,8274986328479596038,2018-09,106430.0,141.0
2,345447888460137901,2018-09,707439.0,99.0
3,1003144834589372198,2018-09,239659.0,96.0
4,324916756972236008,2018-09,203123.0,103.0


In [8]:
df_info.info()
# NO_FUNCIONARIOS possui aproximadamente 5,13% valores nan 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24401 entries, 0 to 24400
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_CLIENTE          24401 non-null  int64  
 1   SAFRA_REF           24401 non-null  object 
 2   RENDA_MES_ANTERIOR  23684 non-null  float64
 3   NO_FUNCIONARIOS     23149 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 762.7+ KB


In [9]:
df_info.describe()

,ID_CLIENTE,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS
count,2.440100e+04,2.368400e+04,23149.000000
mean,4.668214e+18,2.887514e+05,117.799646
std,2.669294e+18,2.115948e+05,21.464574
min,8.784237e+15,1.050000e+02,0.000000
25%,2.347029e+18,1.338662e+05,106.000000
50%,4.795153e+18,2.409985e+05,118.000000
75%,7.061869e+18,3.925018e+05,131.000000
max,9.206031e+18,1.682759e+06,198.000000


In [10]:
# NO_FUNCIONARIOS: média e mediana(50%) estão proximos, indicando valores simétricos
media_funcionarios = df_info['NO_FUNCIONARIOS'].mean()
df_info['NO_FUNCIONARIOS'] = df_info['NO_FUNCIONARIOS'].fillna(int(media_funcionarios))

# RENDA_MES_ANTERIOR: média,mediana(50%) e desvio alto, não estão proximos, indicando valores assimétricos
mediana_renda_anterior = df_info['RENDA_MES_ANTERIOR'].median()
df_info['RENDA_MES_ANTERIOR'] = df_info['RENDA_MES_ANTERIOR'].fillna(int(mediana_renda_anterior))

df_info['SAFRA_REF'] = pd.to_datetime(df_info['SAFRA_REF'], format='%Y-%m').dt.to_period('M') 

In [11]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24401 entries, 0 to 24400
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype    
---  ------              --------------  -----    
 0   ID_CLIENTE          24401 non-null  int64    
 1   SAFRA_REF           24401 non-null  period[M]
 2   RENDA_MES_ANTERIOR  24401 non-null  float64  
 3   NO_FUNCIONARIOS     24401 non-null  float64  
dtypes: float64(2), int64(1), period[M](1)
memory usage: 762.7 KB


In [12]:
df_pagamentos_desenvolvimento.head()

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA
0,1661240395903230676,2018-08,2018-08-17,2018-09-06,2018-09-06,35516.41,6.99
1,1661240395903230676,2018-08,2018-08-19,2018-09-11,2018-09-10,17758.21,6.99
2,1661240395903230676,2018-08,2018-08-26,2018-09-18,2018-09-17,17431.96,6.99
3,1661240395903230676,2018-08,2018-08-30,2018-10-11,2018-10-05,1341.00,6.99
4,1661240395903230676,2018-08,2018-08-31,2018-09-20,2018-09-20,21309.85,6.99


In [13]:
df_pagamentos_desenvolvimento.info()
# VALOR_A_PAGAR possui aproximadamente 1,51% valores nan

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77414 entries, 0 to 77413
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID_CLIENTE              77414 non-null  int64  
 1   SAFRA_REF               77414 non-null  object 
 2   DATA_EMISSAO_DOCUMENTO  77414 non-null  object 
 3   DATA_PAGAMENTO          77414 non-null  object 
 4   DATA_VENCIMENTO         77414 non-null  object 
 5   VALOR_A_PAGAR           76244 non-null  float64
 6   TAXA                    77414 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 4.1+ MB


In [14]:
df_pagamentos_desenvolvimento['SAFRA_REF'] = pd.to_datetime(df_pagamentos_desenvolvimento['SAFRA_REF'], format='%Y-%m').dt.to_period('M') 
df_pagamentos_desenvolvimento['DATA_PAGAMENTO'] = pd.to_datetime(df_pagamentos_desenvolvimento['DATA_PAGAMENTO'])
df_pagamentos_desenvolvimento['DATA_VENCIMENTO'] = pd.to_datetime(df_pagamentos_desenvolvimento['DATA_VENCIMENTO'])

In [15]:
# CRIAÇÃO DO TARGET 
# cálculo de dias de atraso
df_pagamentos_desenvolvimento['DIAS_ATRASO'] = (df_pagamentos_desenvolvimento['DATA_PAGAMENTO'] - df_pagamentos_desenvolvimento['DATA_VENCIMENTO']).dt.days
df_pagamentos_desenvolvimento['INADIMPLENTE'] = (df_pagamentos_desenvolvimento['DIAS_ATRASO'] >= 5).astype(int)

df_pagamentos_desenvolvimento[['DATA_PAGAMENTO','DATA_VENCIMENTO','DIAS_ATRASO', 'INADIMPLENTE']].head()

,DATA_PAGAMENTO,DATA_VENCIMENTO,DIAS_ATRASO,INADIMPLENTE
0,2018-09-06,2018-09-06,0,0
1,2018-09-11,2018-09-10,1,0
2,2018-09-18,2018-09-17,1,0
3,2018-10-11,2018-10-05,6,1
4,2018-09-20,2018-09-20,0,0


In [16]:
df_pagamentos_desenvolvimento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77414 entries, 0 to 77413
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID_CLIENTE              77414 non-null  int64         
 1   SAFRA_REF               77414 non-null  period[M]     
 2   DATA_EMISSAO_DOCUMENTO  77414 non-null  object        
 3   DATA_PAGAMENTO          77414 non-null  datetime64[ns]
 4   DATA_VENCIMENTO         77414 non-null  datetime64[ns]
 5   VALOR_A_PAGAR           76244 non-null  float64       
 6   TAXA                    77414 non-null  float64       
 7   DIAS_ATRASO             77414 non-null  int64         
 8   INADIMPLENTE            77414 non-null  int32         
dtypes: datetime64[ns](2), float64(2), int32(1), int64(2), object(1), period[M](1)
memory usage: 5.0+ MB


In [17]:
df_pagamentos_desenvolvimento.groupby('INADIMPLENTE')['INADIMPLENTE'].count()
# variável target desbalanceada
# provalvemente o modelo não consiga detectar muito bem os valores 1 sendo apenas 7,02%

INADIMPLENTE
0    71978
1     5436
Name: INADIMPLENTE, dtype: int64

In [18]:
df_base = pd.merge(df_pagamentos_desenvolvimento, df_cadastral, on='ID_CLIENTE', how='left')
df_base = pd.merge(df_base, df_info, on=['ID_CLIENTE', 'SAFRA_REF'], how='left')

In [19]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77414 entries, 0 to 77413
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID_CLIENTE              77414 non-null  int64         
 1   SAFRA_REF               77414 non-null  period[M]     
 2   DATA_EMISSAO_DOCUMENTO  77414 non-null  object        
 3   DATA_PAGAMENTO          77414 non-null  datetime64[ns]
 4   DATA_VENCIMENTO         77414 non-null  datetime64[ns]
 5   VALOR_A_PAGAR           76244 non-null  float64       
 6   TAXA                    77414 non-null  float64       
 7   DIAS_ATRASO             77414 non-null  int64         
 8   INADIMPLENTE            77414 non-null  int32         
 9   DATA_CADASTRO           77414 non-null  datetime64[ns]
 10  SEGMENTO_INDUSTRIAL     75997 non-null  object        
 11  DOMINIO_EMAIL           76516 non-null  object        
 12  PORTE                   74938 non-null  object

In [20]:
# tempo de cliente em meses
df_base['TEMPO_CLIENTE_MESES'] = ((df_base['SAFRA_REF'].dt.year - df_base['DATA_CADASTRO'].dt.year) * 12 + (df_base['SAFRA_REF'].dt.month - df_base['DATA_CADASTRO'].dt.month))
# taxa por valor a pagar
df_base['TAXA_JUROS'] = df_base['VALOR_A_PAGAR'] * (df_base['TAXA'] / 100)

df_base[['VALOR_A_PAGAR', 'NO_FUNCIONARIOS', 'TAXA', 'TEMPO_CLIENTE_MESES', 'TAXA_JUROS']].head()

,VALOR_A_PAGAR,NO_FUNCIONARIOS,TAXA,TEMPO_CLIENTE_MESES,TAXA_JUROS
0,35516.41,NaN,6.99,60,2482.597059
1,17758.21,NaN,6.99,60,1241.298879
2,17431.96,NaN,6.99,60,1218.494004
3,1341.00,NaN,6.99,60,93.735900
4,21309.85,NaN,6.99,60,1489.558515


In [21]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77414 entries, 0 to 77413
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID_CLIENTE              77414 non-null  int64         
 1   SAFRA_REF               77414 non-null  period[M]     
 2   DATA_EMISSAO_DOCUMENTO  77414 non-null  object        
 3   DATA_PAGAMENTO          77414 non-null  datetime64[ns]
 4   DATA_VENCIMENTO         77414 non-null  datetime64[ns]
 5   VALOR_A_PAGAR           76244 non-null  float64       
 6   TAXA                    77414 non-null  float64       
 7   DIAS_ATRASO             77414 non-null  int64         
 8   INADIMPLENTE            77414 non-null  int32         
 9   DATA_CADASTRO           77414 non-null  datetime64[ns]
 10  SEGMENTO_INDUSTRIAL     75997 non-null  object        
 11  DOMINIO_EMAIL           76516 non-null  object        
 12  PORTE                   74938 non-null  object

In [22]:
# encoding de variáveis categóricas
le_porte = LabelEncoder()
le_segmento = LabelEncoder()

df_base['PORTE'] = le_porte.fit_transform(df_base['PORTE'].astype(str))
df_base['SEGMENTO_INDUSTRIAL'] = le_segmento.fit_transform(df_base['SEGMENTO_INDUSTRIAL'].astype(str))

In [23]:
recursos = [
    'RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS', 'PORTE', 'CEP_2_DIG',
    'VALOR_A_PAGAR', 'TAXA', 'TAXA_JUROS', 
    'TEMPO_CLIENTE_MESES', 'SEGMENTO_INDUSTRIAL', 'INADIMPLENTE'
]

df_base_recursos = df_base[recursos].dropna()

df_base_recursos.info()
# Aproximadamente 6,5% de nan

<class 'pandas.core.frame.DataFrame'>
Index: 72364 entries, 17 to 77413
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   RENDA_MES_ANTERIOR   72364 non-null  float64
 1   NO_FUNCIONARIOS      72364 non-null  float64
 2   PORTE                72364 non-null  int32  
 3   CEP_2_DIG            72364 non-null  Int64  
 4   VALOR_A_PAGAR        72364 non-null  float64
 5   TAXA                 72364 non-null  float64
 6   TAXA_JUROS           72364 non-null  float64
 7   TEMPO_CLIENTE_MESES  72364 non-null  int64  
 8   SEGMENTO_INDUSTRIAL  72364 non-null  int32  
 9   INADIMPLENTE         72364 non-null  int32  
dtypes: Int64(1), float64(5), int32(3), int64(1)
memory usage: 5.3 MB


In [24]:
df_base.groupby('INADIMPLENTE')['INADIMPLENTE'].count()

INADIMPLENTE
0    71978
1     5436
Name: INADIMPLENTE, dtype: int64

In [25]:
df_base_recursos.groupby('INADIMPLENTE')['INADIMPLENTE'].count()
# 0 aproximadamete 6,42% de perda
# 1 aproximadamete 7,81% de perda

INADIMPLENTE
0    67353
1     5011
Name: INADIMPLENTE, dtype: int64

In [26]:
# TREINAMENDO E TESTE DOS MODELOS

# vou testa os modelos de Regressão Lógistica e Random Forest

# Regressão Lógistica
X = df_base_recursos.drop('INADIMPLENTE', axis=1) 
y = df_base_recursos['INADIMPLENTE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# criar e treinar o modelo
rl = LogisticRegression(random_state=42, class_weight='balanced')
rl.fit(X_train_scaled, y_train)

y_pred = rl.predict(X_test_scaled)

#print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.61      0.75     13471
           1       0.11      0.67      0.19      1002

    accuracy                           0.62     14473
   macro avg       0.54      0.64      0.47     14473
weighted avg       0.90      0.62      0.71     14473



In [27]:
# Random Forest
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# criar e treinar o modelo
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight='balanced')
rf.fit(X_train, y_train)

y_proba = rf.predict_proba(X_test)[:, 1]
y_pred = rf.predict(X_test)

#print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96     13471
           1       0.45      0.72      0.55      1002

    accuracy                           0.92     14473
   macro avg       0.71      0.83      0.75     14473
weighted avg       0.94      0.92      0.93     14473



In [28]:
'''
O modelo Random Forest teve um desempenho melhor que o de Regressão Logística.

Resumo da performance do modelo Random Forest:
Acurácia: 92%.

Precisão da classe 1 (inadimplente): 0.45
Quando o modelo prever inadimplência, ele acerta 45% das vezes.

Recall da classe 1: 0.72
O modelo consegue encontrar 72% dos inadimplentes reais — aceitável para ação preventiva, que é o objetivo do case!
'''

'\nO modelo Random Forest teve um desempenho melhor que o de Regressão Logística.\n\nResumo da performance do modelo Random Forest:\nAcurácia: 92%.\n\nPrecisão da classe 1 (inadimplente): 0.45\nQuando o modelo prever inadimplência, ele acerta 45% das vezes.\n\nRecall da classe 1: 0.72\nO modelo consegue encontrar 72% dos inadimplentes reais — aceitável para ação preventiva, que é o objetivo do case!\n'

In [29]:
# PREVISÃO NO CONJUNTO DE TESTE
df_pagamentos_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12275 entries, 0 to 12274
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID_CLIENTE              12275 non-null  int64  
 1   SAFRA_REF               12275 non-null  object 
 2   DATA_EMISSAO_DOCUMENTO  12275 non-null  object 
 3   DATA_VENCIMENTO         12275 non-null  object 
 4   VALOR_A_PAGAR           12144 non-null  float64
 5   TAXA                    12275 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 575.5+ KB


In [30]:
# PRÉ-PROCESSAMENTO (df_pagamentos_teste)
df_pagamentos_teste['SAFRA_REF'] = pd.to_datetime(df_pagamentos_teste['SAFRA_REF'], format='%Y-%m').dt.to_period('M')  

df_pagamentos_previsao = pd.merge(df_pagamentos_teste, df_info, on=['ID_CLIENTE', 'SAFRA_REF'], how='left')
df_pagamentos_previsao = pd.merge(df_pagamentos_previsao, df_cadastral, on='ID_CLIENTE', how='left')

df_pagamentos_previsao['TEMPO_CLIENTE_MESES'] = ((df_pagamentos_previsao['SAFRA_REF'].dt.year - df_pagamentos_previsao['DATA_CADASTRO'].dt.year) * 12 + (df_pagamentos_previsao['SAFRA_REF'].dt.month - df_pagamentos_previsao['DATA_CADASTRO'].dt.month))
df_pagamentos_previsao['TAXA_JUROS'] = df_pagamentos_previsao['VALOR_A_PAGAR'] * df_pagamentos_previsao['TAXA']

# Encoding igual ao treino
df_pagamentos_previsao['PORTE'] = le_porte.transform(df_pagamentos_previsao['PORTE'].astype(str))
df_pagamentos_previsao['SEGMENTO_INDUSTRIAL'] = le_segmento.transform(df_pagamentos_previsao['SEGMENTO_INDUSTRIAL'].astype(str))

df_pagamentos_previsao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12275 entries, 0 to 12274
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID_CLIENTE              12275 non-null  int64         
 1   SAFRA_REF               12275 non-null  period[M]     
 2   DATA_EMISSAO_DOCUMENTO  12275 non-null  object        
 3   DATA_VENCIMENTO         12275 non-null  object        
 4   VALOR_A_PAGAR           12144 non-null  float64       
 5   TAXA                    12275 non-null  float64       
 6   RENDA_MES_ANTERIOR      11861 non-null  float64       
 7   NO_FUNCIONARIOS         11861 non-null  float64       
 8   DATA_CADASTRO           12237 non-null  datetime64[ns]
 9   SEGMENTO_INDUSTRIAL     12275 non-null  int32         
 10  DOMINIO_EMAIL           12067 non-null  object        
 11  PORTE                   12275 non-null  int32         
 12  CEP_2_DIG               12236 non-null  Int64 

In [31]:
recursos.remove('INADIMPLENTE')

# Previsão
X_test = df_pagamentos_previsao[recursos]
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12275 entries, 0 to 12274
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   RENDA_MES_ANTERIOR   11861 non-null  float64
 1   NO_FUNCIONARIOS      11861 non-null  float64
 2   PORTE                12275 non-null  int32  
 3   CEP_2_DIG            12236 non-null  Int64  
 4   VALOR_A_PAGAR        12144 non-null  float64
 5   TAXA                 12275 non-null  float64
 6   TAXA_JUROS           12144 non-null  float64
 7   TEMPO_CLIENTE_MESES  12237 non-null  float64
 8   SEGMENTO_INDUSTRIAL  12275 non-null  int32  
dtypes: Int64(1), float64(6), int32(2)
memory usage: 779.3 KB


In [32]:
df_pagamentos_previsao['PROBABILIDADE_INADIMPLENCIA'] = rf.predict_proba(X_test)[:, 1]

df_submissao = df_pagamentos_previsao[['ID_CLIENTE', 'SAFRA_REF', 'PROBABILIDADE_INADIMPLENCIA']]

In [33]:
df_submissao.head()

,ID_CLIENTE,SAFRA_REF,PROBABILIDADE_INADIMPLENCIA
0,5058298901476893676,2021-07,0.152451
1,274692171162531764,2021-07,0.322327
2,274692171162531764,2021-07,0.322327
3,274692171162531764,2021-07,0.322327
4,465309249432033993,2021-07,0.338526


In [34]:
df_submissao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12275 entries, 0 to 12274
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype    
---  ------                       --------------  -----    
 0   ID_CLIENTE                   12275 non-null  int64    
 1   SAFRA_REF                    12275 non-null  period[M]
 2   PROBABILIDADE_INADIMPLENCIA  12275 non-null  float64  
dtypes: float64(1), int64(1), period[M](1)
memory usage: 287.8 KB


In [35]:
df_submissao.to_csv(f'{caminho_arquivo}submissao_case.csv', index=False, sep=';', encoding='utf-8')